In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
from pathlib import Path
from PIL import Image
from tqdm import tqdm
import imageio.v2 as imageio
from io import BytesIO
from skimage.morphology import footprint_rectangle

from skimage.filters import gaussian, threshold_local
from scipy.ndimage import binary_closing, binary_opening

from scipy.signal import convolve2d
from skimage.morphology import skeletonize
from skimage.filters import threshold_local

import networkx as nx
from scipy.ndimage import convolve
from sklearn.neighbors import KDTree
from scipy import ndimage
from skimage import morphology
from skimage.filters import threshold_otsu
class ReactionDiffusionModel:
    def __init__(self, mode, k, c, e):
        self.mode = mode
        self.N = 100
        self.treal = 20 if mode != 'doubling' else 27
        self.dt = 0.5e-7
        self.Lreal = 1

        # Параметры модели
        self.k = k
        self.c = c
        self.e = e


        self.U0 = 1.1
        self.V0 = 2
        self.U_fl = 0.02
        self.V_fl = 0.02

        # Параметры диффузии
        self.Du_r = 0.15e-8
        self.Dv_r = 30e-8
        self.D0 = 15e-8
        self.Du = self.Du_r/self.D0
        self.Dv = self.Dv_r/self.D0

        # Геометрия и время
        self.L0 = 4
        self.L = self.Lreal/self.L0
        self.dx = self.L / self.N
        self.Tc = 3600
        self.t0 = (self.L0**2)/self.D0
        self.tmod = self.treal*24*3600/(self.t0)
        self.total_steps = int(self.tmod/self.dt)
        self.gamma = self.t0/self.Tc

        # Настройки для разных режимов

        if self.mode == 'doubling':
            self.gamma = self.gamma*2

        self.S0 = 2.0 if mode == 'spots' else 0.0
        self.S = np.full((self.N, self.N), self.S0)
        self.S_increment = 0.006


        # Инициализация полей
        np.random.seed(3425)
        self.U = self.U0 * np.ones((self.N, self.N))
        self.V = self.V0 * np.ones((self.N, self.N))
        # self.U += self.U_fl * self.U0 * np.random.random(size=(self.N, self.N))
        # self.V += self.V_fl * self.V0 * np.random.random(size=(self.N, self.N))
        self.U = self.U0 + self.U_fl  * self.U0 * (2 * np.random.random((self.N, self.N)) - 1)
        self.V = self.V0 + self.V_fl * self.V0 * (2 * np.random.random((self.N, self.N)) - 1)

    def laplacian(self, Z):
        Ztop = Z[0:-2, 1:-1]
        Zleft = Z[1:-1, 0:-2]
        Zbottom = Z[2:, 1:-1]
        Zright = Z[1:-1, 2:]
        Zcenter = Z[1:-1, 1:-1]
        return (Ztop + Zleft + Zbottom + Zright - 4*Zcenter) / (self.dx**2 + 1e-10)

    def reaction_terms(self, U, V):
        U_sq = U[1:-1,1:-1]**2
        V_safe = np.clip(V[1:-1,1:-1], 1e-10, None)
        f = self.gamma * (U_sq / ((1 + self.k*U_sq)*V_safe) - self.c*U[1:-1,1:-1])
        g = self.gamma * (U_sq - self.e*V[1:-1,1:-1] + self.S[1:-1,1:-1])
        return f, g

    def rk2_step(self):
        # Шаг 1: Предиктор
        f1, g1 = self.reaction_terms(self.U, self.V)
        lap_U1 = self.laplacian(self.U)
        lap_V1 = self.laplacian(self.V)

        U1 = self.U.copy()
        V1 = self.V.copy()
        U1[1:-1, 1:-1] += self.dt * (self.Du * lap_U1 + f1)
        V1[1:-1, 1:-1] += self.dt * (self.Dv * lap_V1 + g1)

        # Граничные условия
        for Z in [U1, V1]:
            Z[0, :] = Z[1, :]; Z[-1, :] = Z[-2, :]
            Z[:, 0] = Z[:, 1]; Z[:, -1] = Z[:, -2]

        # Шаг 2: Корректор
        f2, g2 = self.reaction_terms(U1, V1)
        lap_U2 = self.laplacian(U1)
        lap_V2 = self.laplacian(V1)

        self.U[1:-1, 1:-1] += 0.5 * self.dt * (self.Du * (lap_U1 + lap_U2) + (f1 + f2))
        self.V[1:-1, 1:-1] += 0.5 * self.dt * (self.Dv * (lap_V1 + lap_V2) + (g1 + g2))

        for Z in [self.U, self.V]:
            Z[0, :] = Z[1, :]; Z[-1, :] = Z[-2, :]
            Z[:, 0] = Z[:, 1]; Z[:, -1] = Z[:, -2]


    # Улучшенный метод контрастирования с адаптивной бинаризацией
    def _enhance_contrast(self, data):
        filtered = ndimage.median_filter(data, size=3)
        p5, p95 = np.percentile(filtered, [5, 95])
        normalized = (filtered - p5) / (p95 - p5 + 1e-10)
        normalized = np.clip(normalized, 0, 1)

        # Адаптивная бинаризация
        block_size = 41
        local_thresh = threshold_local(normalized, block_size=block_size)
        binary = normalized > local_thresh

        # Морфологическая обработка
        binary = morphology.binary_opening(binary, footprint=np.ones((2,2)))
        binary = morphology.remove_small_objects(binary, min_size=8)
        binary = morphology.remove_small_holes(binary, area_threshold=8)

        return  binary

    def run_simulation(self):
        for step in tqdm(range(self.total_steps), desc=f"{self.mode} progress"):
            if self.mode == 'spots' and step % (self.total_steps/1000) == 0 and step > 0:
                self.S += self.S_increment
                current_S_avg = np.mean(self.S)
                print(f"\nШаг {step}: увеличено S (среднее по матрице {current_S_avg:.6f})")
            self.rk2_step()

        enhanced = self._enhance_contrast(self.U)

        binary = enhanced > threshold_local(enhanced, block_size=41)
        return self.U, binary

if __name__ == "__main__":
    # Определяем диапазоны параметров
    k_values = [0.02, 0.03]
    c_values = [ 0.01, 0.02]
    e_values = [0.04, 0.06]

    # Перебираем все комбинации параметров
    for k in k_values:
        for c in c_values:
            for e in e_values:
                # Создаем фигуру для вывода результатов
                plt.figure(figsize=(15, 10))
                plt.suptitle(f'Parameters: k={k}, c={c}, e={e}', fontsize=16, y=1.05)

                # Последовательно выполняем все три режима
                for i, mode in enumerate(['stripes', 'spots', 'doubling'], 1):
                    model = ReactionDiffusionModel(mode, k, c, e)
                    pattern, binary_pattern = model.run_simulation()

                    # Отображаем оригинальные паттерны (верхний ряд)
                    plt.subplot(2, 3, i)
                    plt.imshow(pattern, cmap='binary')
                    plt.title(f'Original: {mode}')
                    plt.axis('tight')
                    plt.axis('off')

                    # Отображаем бинаризованные паттерны (нижний ряд)
                    plt.subplot(2, 3, i+3)
                    plt.imshow(binary_pattern, cmap='binary')
                    plt.title(f'Binary: {mode}')
                    plt.axis('tight')
                    plt.axis('off')

                plt.tight_layout()
                plt.show()